In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import *
from pandas import read_csv
from pandas import ExcelFile
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

In [2]:
def train_model(data):
    
    #get data
    Y=data['UpDownSign']
    X=data.drop(columns=['UpDownSign','AdjClose'])
    
    #Split the data to train and test
    Xtrain,Xtest,ytrain,ytest = train_test_split(X,Y, random_state = 17)
    
    '''
    #Below uses K-Fold Validation method to find the best parameters
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 5)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt','log2']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(5, 100, num = 15)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,}
    
    rf = RandomForestClassifier()
    regr = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=41)
    regr.fit(Xtrain,ytrain)
    
    print(regr.best_params_)
    '''
    
    '''
    regr_best=RandomForestClassifier(n_estimators=300,min_samples_split=2,min_samples_leaf=4,max_features='log2',max_depth=50,random_state=17,oob_score=True)
    regr_best.fit(Xtrain,ytrain)
    ypred=regr_best.predict(Xtest)
    
    #print accuracy rate
    print('Accuracy Score:',accuracy_score(ytest, ypred, normalize=True)*100.0)
    
    #print OOB score (R^2 for out-of-bag sample set)
    print('OOB score',regr_best.oob_score_)
    '''
    regr_best=RandomForestClassifier(n_estimators=700,min_samples_split=5,min_samples_leaf=2,max_features='log2',max_depth=40,random_state=17,oob_score=True)
    regr_best.fit(Xtrain,ytrain)
    ypred=regr_best.predict(Xtest)
    
    '''
    #print accuracy rate
    print('Accuracy Score:',accuracy_score(ytest, ypred, normalize=True)*100.0)
    
    #print OOB score (R^2 for out-of-bag sample set)
    print('OOB score',regr_best.oob_score_)
    
    #print report metrics
    report = classification_report(ytest, ypred)
    print(report)
    
    #print feature significance
    feature_significance=pd.DataFrame(regr_best.feature_importances_)
    feature_significance.index=X.columns
    feature_significance.rename(columns={0:"significance score"},inplace=True)
    feature_significance=feature_significance.sort_values(by='significance score',ascending=False)
    
    plt.rcParams["font.family"] = "Times New Roman"
    plt.rcdefaults()
    fig, ax = plt.subplots()
    y_pos = np.arange(feature_significance.index.size)
    ax.barh(feature_significance.index, feature_significance['significance score'])
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feature_significance.index)
    ax.invert_yaxis()
    ax.set_xlabel('Signicance Score')
    plt.show()
    '''
    '''
    #Below is an example of decision tree
    estimator = regr_best.estimators_[100]
    
    # Export as dot file
    export_graphviz(estimator, out_file='tree.dot', 
                feature_names = X.columns,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

    # Convert to png using system command (requires Graphviz)
    call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

    # Display in jupyter notebook
    Image(filename = 'tree.png')
    '''
    
    return regr_best